In [ ]:
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import os
import importlib

from ATARI.sammy_interface import sammy_classes, sammy_functions

from ATARI.ModelData.particle_pair import Particle_Pair
from ATARI.ModelData.experimental_model import Experimental_Model

from copy import copy, deepcopy

import ATARI.utils.plotting as myplot


In [ ]:
sammypath = ''
sammypath = "/Users/noahwalton/gitlab/sammy/sammy/build/bin/sammy"
assert(sammypath != '')

In [ ]:
%matplotlib widget

# Overview

The synthetic data methodology build here is modeled after measurement data from the work of Jesse Brown at RPI (currently at ORNL).
These measurements are detailed in https://www.tandfonline.com/doi/full/10.1080/00295639.2023.2249786 and consist of 6 experimental configurations.
This notebook investigates the 1, 3, and 6 mm transmission measurements.

## 1, 3, & 6 mm Transmission Measurements

For these measurement models, we do not have access to the raw data or the full data-reduction model like we did for the 12mm validation case.
Instead, we only have access to the published journal article for the measurements, an appropriate resolution function, and the reduced data in EXFOR.
We must glean the necessary information for the ATARI.syndat measurement models from these sources.

In [ ]:
from ATARI.ModelData.measurement_models.transmission_rpi import Transmission_RPI
from ATARI.syndat.syndat_model import Syndat_Model
from ATARI.syndat.data_classes import syndatOPT

In [ ]:

### Determine channel widths
from ATARI.theory.experimental import e_to_t, t_to_e

def get_chw_and_upperE(E, FP):
    E = np.array(E)
    tof = e_to_t(E, FP, True)
    dt = np.diff(tof*1e6)
    widths1, index1 = np.unique(np.round(dt, 4), return_index=True)
    chw, Emax = np.flipud(widths1), np.flipud(E[index1])
    strc = ''
    stre = ''
    for c,e in zip(chw, Emax):
        strc += f"{c*1e3:.2f}, "
        stre += f"{e:.2f}, "
    return stre, strc



energy_range_T = [0.15e3, 100e3]
# energy_range_T = [0.15e3, 2.5e3]


### 1mm Transmission data
transdat1 = sammy_functions.readlst(os.path.realpath("./data/trans-Ta-1mm.twenty"))
transdat1_covfile = os.path.realpath('./data/trans-Ta-1mm.idc')

chw, Emax = get_chw_and_upperE(transdat1.E, 100.13)

exptrans1 = Experimental_Model(title = "trans1mm",
                                reaction = "transmission", 
                                energy_range = energy_range_T,
                                energy_grid = np.sort(transdat1.E.values),
                                template = os.path.realpath(f'./data/template_trans1mm_edited'),
                                n = (0.00566,0.0),  
                                FP = (100.13,0.0), 
                                burst = (8, 0.0), 
                                temp = (294.2610, 0.0),
                                t0 = (7614.9, 0.0),

                               channel_widths={
                                    "maxE": [216.16, 613.02, 6140.23], 
                                    "chw": [204.7, 102.4, 51.2],
                                    "dchw": [1.6, 1.6, 1.6]
                                }
                                
                               )
transdat1 = transdat1.loc[(transdat1.E<max(exptrans1.energy_range)) & (transdat1.E>min(exptrans1.energy_range)), :]

### 3mm transmission data
transdat3 = sammy_functions.readlst(os.path.realpath("./data/trans-Ta-3mm.twenty"))
transdat3_covfile = os.path.realpath("./data/trans-Ta-3mm.idc")

exptrans3 = Experimental_Model(title = "trans3mm",
                               reaction = "transmission", 
                               energy_range = energy_range_T,
                                energy_grid = np.sort(transdat3.E.values),
                                template = os.path.realpath(f'./data/template_trans3mm_edited'),
                                n = (0.017131,0.0),  
                                FP = (100.13,0.0), 
                                burst = (8, 0.0), 
                                temp = (294.2610, 0.0),
                                t0 = (7614.9, 0.0),

                               channel_widths={
                                    "maxE": [216.16, 613.02, 6140.23], 
                                    "chw": [204.7, 102.4, 51.2],
                                    "dchw": [1.6, 1.6, 1.6]
                                }
                                
                               )
transdat3 = transdat3.loc[(transdat3.E<max(exptrans3.energy_range)) & (transdat3.E>min(exptrans3.energy_range)), :]


### 6mm transmission data
transdat6 = sammy_functions.readlst(os.path.realpath("./data/trans-Ta-6mm.twenty"))
transdat6_covfile = os.path.realpath("./data/trans-Ta-6mm.idc")

exptrans6 = Experimental_Model(title = "trans6mm",
                               reaction = "transmission", 
                               energy_range = energy_range_T,
                                energy_grid = np.sort(transdat6.E.values),
                                template = os.path.realpath(f'./data/template_trans6mm_edited'),
                                n = (0.03356,0.0),  
                                FP = (100.13,0.0), 
                                burst = (8, 0.0), 
                                temp = (294.2610, 0.0),
                                t0 = (7614.9, 0.0),

                               channel_widths={
                                    "maxE": [216.16, 613.02, 6140.23], 
                                    "chw": [204.7, 102.4, 51.2],
                                    "dchw": [1.6, 1.6, 1.6]
                                }
                                
                               )
transdat6 = transdat6.loc[(transdat6.E<max(exptrans6.energy_range)) & (transdat6.E>min(exptrans6.energy_range)), :]

In [ ]:
importlib.reload(myplot)
### setup in zipped lists 
datasets = [transdat1, transdat3, transdat6]
experiments= [exptrans1, exptrans3, exptrans6]

covariance_data = [transdat1_covfile, transdat3_covfile, transdat6_covfile]

fig = myplot.plot_reduced_data_T(datasets, experiments,
                                 xlim=energy_range_T, #(.15e3, 2.5e3),
                               fig_size=(8,3))
fig.suptitle("Reduced Evaluation Data")
fig.tight_layout()
# fig_trans_real.show()


### Approximate the resonances that generated this data

Synthetic data can be generated around any point-wise vector (of the experimental object transmission or capture yield) that is taken as 'true'.
In order to better compare synthetic data to real data, we must approximate the resonance structures that generated the real data and use those as 'true'
For this, you can take the evaluated JEFF parameters or fit the new data from those parameters.
Because we have multiple datasets, the sammy_YW scheme can be used to fit all 3 simultaneously.


In [ ]:
rto = sammy_classes.SammyRunTimeOptions(sammypath,
                             **{"Print"   :   True,
                              "bayes"   :   True,
                              "keep_runDIR"     : True,
                              "sammy_runDIR"    : "sammy_runDIR_123mmT"
                              })

matnum = 7328
jeff_file = os.path.realpath("./data/73-Ta-181g.jeff33")
jeff_parameters = sammy_functions.get_endf_parameters(jeff_file, matnum, rto)

# energy_range_all = (4, 2500)
jeff_parameters = jeff_parameters[(jeff_parameters.E<max(energy_range_T)) & (jeff_parameters.E>min(energy_range_T))]
jeff_parameters["varyGn1"] = np.ones(len(jeff_parameters))
jeff_parameters["varyGg"] = np.ones(len(jeff_parameters))*1
jeff_parameters["varyE"] = np.ones(len(jeff_parameters))


### setup the reaction model - Ta-181 is the default
Ta_pair = Particle_Pair()      

sammyINPyw = sammy_classes.SammyInputDataYW(
    Ta_pair,
    resonance_ladder=jeff_parameters,

    datasets = datasets ,
    experiments= experiments,
    experimental_covariance = covariance_data,
    measurement_models= [trans1mm_gen_meas, trans3mm_gen_meas, trans6mm_gen_meas],
    idc_at_theory=True,

    max_steps = 5,
    iterations = 2,
    step_threshold = 0.01,
    autoelim_threshold = None,

    LS = False,
    LevMar = False,
    LevMarV = 1.5,

    minF = 1e-5,
    maxF = 10,
    initial_parameter_uncertainty = 0.1
)

sammyOUT = sammy_functions.run_sammy_YW(sammyINPyw, rto)
print(sammyOUT.chi2)
print(sammyOUT.chi2_post)

In [ ]:
fig = myplot.plot_reduced_data_T(sammyINPyw.datasets,
                           sammyINPyw.experiments, 
                           fits = sammyOUT.pw_post, 
                           xlim=(150,1000))
fig.tight_layout()

In [ ]:
len(sammyOUT.pw[2])

In [ ]:


# open_dataframe_syndat  = copy(open_dataframe)
# open_dataframe_syndat['tof'] = open_dataframe_syndat['tof']*1e3
# open_dataframe_syndat


open_dataframe_1mm = pd.read_csv("/Users/noahwalton/research_local/resonance_fitting/synthetic_data/Ta181/JB_sol/1mm-100m-opendf.csv")
open_dataframe_3mm = pd.read_csv("/Users/noahwalton/research_local/resonance_fitting/synthetic_data/Ta181/JB_sol/3mm-100m-opendf.csv")
open_dataframe_6mm = pd.read_csv("/Users/noahwalton/research_local/resonance_fitting/synthetic_data/Ta181/JB_sol/6mm-100m-opendf.csv")
# bw = 0.0064  # us
# t0 = 7.6149   # us7760.672247321694
# FP = 100.130  # m  t0 and FP from Uranium

def update_df(df):
    df = copy(df)
    df['tof'] = df['tof'] * 1e3 # syndat takes tof in nanoseconds
    df['bin_width'] = df['bin_width'] * 1e-6 # syndat takes tof in nanoseconds
    df['E'] = t_to_e((df['tof']-7.6149*1e3)*1e-9, 100.13, True) 
    df = df[(df.E<max(energy_range_T)) & (df.E>min(energy_range_T))]# #
    df.rename(columns={"counts" :   "ct",
                       "dcounts" :   "dct",
                       "cps"    :   "c", 
                       "dcps"    :   "dc", 
                       "bin_width"  : "bw"
                       }, inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.drop(columns="Unnamed: 0", inplace=True)
    return df

open_dataframe_1mm = update_df(open_dataframe_1mm)
open_dataframe_3mm = update_df(open_dataframe_3mm)
open_dataframe_6mm = update_df(open_dataframe_6mm)

# open_dataframe_1mm['tof'] = np.flipud(exptrans1.tof_grid)
# open_dataframe_3mm['tof'] = np.flipud(exptrans3.tof_grid)
# open_dataframe_6mm['tof'] = np.flipud(exptrans6.tof_grid)

# open_dataframe_1mm['E'] = np.flipud(exptrans1.energy_grid)
# open_dataframe_3mm['E'] = np.flipud(exptrans3.energy_grid)
# open_dataframe_6mm['E'] = np.flipud(exptrans6.energy_grid)

open_dataframe_1mm

In [ ]:
exptrans1.tof_grid

In [ ]:
print(len(exptrans1.energy_grid))
print(len(open_dataframe_1mm))

print(len(exptrans3.energy_grid))
print(len(open_dataframe_3mm))

print(len(exptrans6.energy_grid))
print(len(open_dataframe_6mm))

In [30]:
from ATARI.syndat import syndat_model
from ATARI.ModelData.measurement_models.transmission_rpi import Transmission_RPI
from ATARI.syndat import control 
importlib.reload(control)
importlib.reload(myplot)


trigo = 1.7e7 # 19919923.0 #
ko, dko = 1.8821644247468616, 0.04334047072747 #1.8821644247468616*0.026 #
ntrigs=0

par_1mmT = {
"trigo" :   (trigo,  0),
"trigs" :   (29089947.0-ntrigs,  0),
"m1"    : (1, 0.01), 
"m2"    : (1, 0.01), 
"m3"    : (1, 0.01), 
"m4"    : (1, 0.01), 
"ks"    : (1.6530519447406526, 0.02413126885507), 
"ko"    : (ko, dko), 
"b0s"   : (123.95, 0.82), 
"b0o"   : (126.48, 1.0), 

"a_b"   :  ([2.62700984e+04, 8.95034813e-01], 
            [[3.834911600262e+02**2, 9.930753594644e-01*3.834911600262e+02*6.005157275119e-02], [9.930753594644e-01*3.834911600262e+02*6.005157275119e-02,  6.005157275119e-02**2]])
}
trans1mm_gen_meas = Transmission_RPI(**par_1mmT, 
                                    open_neutron_spectrum = open_dataframe_1mm,
                                    bkg_func = 'power')
trans1mm_red_meas = Transmission_RPI(**par_1mmT, 
                                    open_neutron_spectrum = open_dataframe_1mm,
                                    bkg_func = 'power')

par_3mmT = {
"trigo"     : (trigo, 0),
"trigs"     : (29399939.0-ntrigs, 0),
"m1"    : (1, 0.01),
"m2"    : (1, 0.01),
"m3"    : (1, 0.01),
"m4"    : (1, 0.01),
"ks"    : (1.518791552772295, 0.02196154844009),
"ko"    : (ko, dko), 
"b0s"   : (120.98, 0.8),
"b0o"   : (126.48, 1.0),
"a_b"   : ([2.62700984e+04, 8.95034813e-01], 
           [[3.834911600262e+02**2, 9.930753594644e-01*3.834911600262e+02*6.005157275119e-02], [9.930753594644e-01*3.834911600262e+02*6.005157275119e-02, 6.005157275119e-02**2]]),
}

trans3mm_gen_meas = Transmission_RPI(**par_3mmT, 
                                    open_neutron_spectrum = open_dataframe_3mm,
                                    bkg_func = 'power')
trans3mm_red_meas = Transmission_RPI(**par_3mmT, 
                                    open_neutron_spectrum = open_dataframe_3mm,
                                    bkg_func = 'power')


par_6mmT = {
"trigo"     : (trigo, 0),
"trigs"     : (29393875.0-ntrigs, 0),
"m1"    : (1, 0.01),
"m2"    : (1, 0.01),
"m3"    : (1, 0.01),
"m4"    : (1, 0.01),
"ks"    : (1.3673393800915357, 0.02287857111889),
"ko"    : (ko, dko), 
"b0s"   : (116.40, 0.79),
"b0o"   : (126.48, 1.0),
"a_b"   : ([2.62700984e+04, 8.95034813e-01], 
           [[4.395560632573e+02**2, 9.930753594644e-01*4.395560632573e+02*6.005157275119e-02], [9.930753594644e-01*4.395560632573e+02*6.005157275119e-02, 6.005157275119e-02**2]])
}
trans6mm_gen_meas = Transmission_RPI(**par_6mmT, 
                                    open_neutron_spectrum = open_dataframe_6mm,
                                    bkg_func = 'power')
trans6mm_red_meas = Transmission_RPI(**par_6mmT, 
                                    open_neutron_spectrum = open_dataframe_6mm,
                                    bkg_func = 'power')

In [ ]:
### define syndat options and model, do not sample resonances or true model parameters while trying to match noise 
synOPT = syndatOPT(sampleRES=False, sampleTMP=True, sampleTNCS=False, sample_counting_noise=True,
                           calculate_covariance=True, explicit_covariance=True, save_raw_data=True)

syndat_trans1mm = Syndat_Model(exptrans1,
                            trans1mm_gen_meas,
                            trans1mm_red_meas,
                            options = synOPT,
                            title='trans1mm')
syndat_trans3mm = Syndat_Model(exptrans3,
                            trans3mm_gen_meas,
                            trans3mm_red_meas,
                            options = synOPT,
                            title='trans3mm')
syndat_trans6mm = Syndat_Model(exptrans6,
                            trans6mm_gen_meas,
                            trans6mm_red_meas,
                            options = synOPT,
                            title='trans6mm')

syndat_models = [syndat_trans1mm, syndat_trans3mm, syndat_trans6mm]

### set the resonance ladder for the reaction model
Ta_pair.resonance_ladder = copy(sammyOUT.par_post)

da_avg = (3.834911600262e+02 + 3.834911600262e+02 + 4.395560632573e+02)/3
model_correlations = [
                    {'models': [1,1,1],
                    # 'a_b'   : ([2.62700984e+04, 8.95034813e-01], 
                    #             [[da_avg**2, 9.930753594644e-01*da_avg*6.005157275119e-02], [9.930753594644e-01*da_avg*6.005157275119e-02, 6.005157275119e-02**2]]),
                    "ko"    : (ko, dko),#0.04334047072747),
                    "b0o"   : (126.48, 1.0),        }

                        ]

### build syndat control module and draw samples
syndat = control.Syndat_Control(particle_pair= Ta_pair,
                        syndat_models = syndat_models,
                        model_correlations = model_correlations,
                        sampleRES = False,
                        save_covariance=True,
                        save_raw_data=True
                        )

# pass pw true instead of recalculating model bc it is expensive ofer this energy range
pw_true_list = deepcopy(sammyOUT.pw_post)
[df.rename(columns={"theo_trans":'true'}, inplace=True) for df in pw_true_list]


rto.bayes = False
syndat.sample(rto, num_samples=100, pw_true_list=pw_true_list)

In [ ]:
sample1 = syndat.get_sample(0)

red_data = sample1['trans3mm'].pw_reduced
raw_data = sample1['trans3mm'].pw_raw

# plot transmission
fig, axes = subplots(1,2, figsize=(12,4))

axes[0].errorbar(red_data.E, red_data.exp, yerr=red_data.exp_unc,
                  fmt='.', alpha=0.5, color='k', linewidth=1.0, markersize=4, capsize=1, label=exptrans1.title)
axes[0].plot(red_data.E, red_data.true, color='r', zorder=1)

axes[0].set_xscale('log')
# axes[0].set_yscale('log')
axes[0].set_ylim([-0.2, 1.2])
axes[0].set_xlabel("Energy (eV)")
axes[0].set_title("Reduced Data for 12mm T")
axes[0].set_ylabel("Transmission")


# axes[1].plot(raw_data.tof, raw_data.cto_true, label='co')
axes[1].plot(raw_data.tof, raw_data.co, label='co')
axes[1].plot(raw_data.tof, raw_data.bo, label='bo')

# axes[1].plot(raw_data.tof, raw_data.cts, label='cs')
axes[1].plot(raw_data.tof, raw_data.cs, label='cs')
axes[1].plot(raw_data.tof, raw_data.bs, label='bs')

axes[1].set_xscale('log')
axes[1].set_yscale('log')

axes[1].set_xlabel("Time-of-flight (us)")
axes[1].set_ylabel("Count Rate (c/s)")
axes[1].set_title("Raw Data for 12mm T")


# axes[2].imshow(trans12_explicit_covariance)
# imshow(trans12_explicit_covariance, vmin=-.1, vmax=.1)

# axes[2].colorbar()
axes[1].legend()

fig.tight_layout()

In [ ]:

syndat_datasets = [out.pw_reduced for key, out in sample1.items()]
fig_synthetic = myplot.plot_reduced_data_T(syndat_datasets[:], experiments[:],
                                        xlim=(310, 400),#energy_range_T, #(.15e3, 2.5e3),
                                    fig_size=(8,3),
                                    fits=sammyOUT.pw_post)

fig_trans_real = myplot.plot_reduced_data_T(datasets[:], experiments[:],
                                 xlim=(310, 400),#energy_range_T
                               fig_size=(8,3),
                               fits=sammyOUT.pw_post)


### Comparing how the data is distributed

The following distributions are constructed after filtering the data to areas of the RRR that are well-represented by the fitted cross section (150< eV < 850).


In [ ]:

mask_syndat = (syndat_datasets[0].E>310.0) & (syndat_datasets[0].E<400)
mask_real= (datasets[0].E>310.0) & (datasets[0].E<400)

for syn, real in zip(syndat_datasets, datasets):
# syn = syndat_datasets[0]
# real = datasets[0]

    synthetic_data = syn[mask_syndat]
    real_data = real[mask_real]


    fig, axes = subplots(1,3, figsize=(12,3))

    axes[0].scatter(synthetic_data.exp, synthetic_data.exp_unc**2, label='synthetic')
    axes[0].scatter(real_data.exp, real_data.exp_unc**2, label='real', alpha=0.5)
    axes[0].set_xlim([-0.1,1.2])
    axes[0].set_ylim([-0.1, 0.2])
    axes[0].set_ylabel("Uncertainty Value")
    axes[0].set_xlabel("Transmission Value")

    bins = 150
    hist_range = (-4,4)
    #/synthetic_data.exp_unc
    #/real_data.exp_unc.values
    _ = axes[1].hist((synthetic_data.exp-synthetic_data.true)/synthetic_data.exp_unc, bins=bins, range=hist_range, density=True,label='synthetic') #-red_data.true
    _ = axes[1].hist((real_data.exp.values-np.flipud(synthetic_data.true.values))/real_data.exp_unc.values, bins=bins, range=hist_range, alpha=0.5,density=True, label='real')#-red_data.true
    axes[1].set_xlabel("Normalized Residuals")
    # range = (-0.1, 1.2)
    # _ = axes[1].hist((synthetic_data.exp), bins=bins, range=range, density=True,label='synthetic')  #-red_data.true
    # _ = axes[1].hist((real_data.exp), bins=bins, range=range, alpha=0.5,density=True, label='real') #-red_data.true
    # axes[1].set_xlabel("Transmission Values")
    # axes[1].set_ylabel("Frequency")


    bins = 150
    hist_range = (-.01,.15)
    _ = axes[2].hist(synthetic_data.exp_unc, bins=bins, range=hist_range, density=True,label='synthetic')
    _ = axes[2].hist(real_data.exp_unc, bins=bins, range=hist_range, alpha=0.5, density=True,label='real')
    axes[2].set_xlabel("Uncertainty")
    axes[2].set_ylabel("Frequency")

    for ax in axes:
        ax.legend()

    # fig.suptitle("")
    fig.tight_layout()

In [ ]:
synthetic_residual = []
for i in range(100):
    sample1 = syndat.get_sample(i)
    syndat_datasets = [out.pw_reduced for key, out in sample1.items()]


    mask_syndat = (syndat_datasets[0].E>310.0) & (syndat_datasets[0].E<400)
    mask_real= (datasets[0].E>310.0) & (datasets[0].E<400)

    # for syn, real in zip(syndat_datasets, datasets):
    syn, real = syndat_datasets[0], datasets[0]

    synthetic_data = syn[mask_syndat]
    real_data = real[mask_real]

    # fig, axes = subplots(1,3, figsize=(12,3))

    # axes[0].scatter(synthetic_data.exp, synthetic_data.exp_unc, label='synthetic')
    # axes[0].scatter(real_data.exp, real_data.exp_unc, label='real', alpha=0.5)

    synthetic_residual.append((synthetic_data.exp-synthetic_data.true)/synthetic_data.exp_unc)

real_residual = (real_data.exp.values-np.flipud(synthetic_data.true.values))/real_data.exp_unc.values

    # _ = axes[2].hist(synthetic_data.exp_unc, bins=bins, range=range, density=True,label='synthetic')
    # _ = axes[2].hist(real_data.exp_unc, bins=bins, range=range, alpha=0.5, density=True,label='real')

In [ ]:
figure()
_ = hist(np.array(synthetic_residual).flatten(), bins=bins, density=True, )
_ = hist(real_residual, bins=bins, density=True, alpha=0.5)


# Saving the Syndat model
The syndat_model and syndat_control objects can be saved using pickle.
All parameters and settings will follow such that the model can be loaded and immediately sampled from.
If saving the syndat_model objects, samples generated using syndat_control must be cleared.
This is the preferred method for this notebook because these syndat models are going to be compiled and sampled from elsewhere (in GenerateData_Ta181.ipynb).
Alternatively, you can save the control object.
This will allow all of the samples to also be saved in the pkl.

In [ ]:
from ATARI.utils import atario
importlib.reload(atario)

### saving is commented out because the models have already been generated and saved
# atario.save_syndat_model(syndat_trans1mm, os.path.join(os.getcwd(), "results", "SyndatModel_1mmT.pkl"), clear_samples=True)
# atario.save_syndat_model(syndat_trans3mm, os.path.join(os.getcwd(), "results", "SyndatModel_3mmT.pkl"), clear_samples=True)
# atario.save_syndat_model(syndat_trans6mm, os.path.join(os.getcwd(), "results", "SyndatModel_6mmT.pkl"), clear_samples=True)

In [26]:
# atario.save_syndat_model(syndat_trans1mm, os.path.join("/Users/noahwalton/research_local/resonance_fitting/ATARI_workspace/Ta181_Syndat_150eV_100keV/syndat_models_E150_100e3", "SyndatModel_1mmT.pkl"), clear_samples=True)
# atario.save_syndat_model(syndat_trans3mm, os.path.join("/Users/noahwalton/research_local/resonance_fitting/ATARI_workspace/Ta181_Syndat_150eV_100keV/syndat_models_E150_100e3", "SyndatModel_3mmT.pkl"), clear_samples=True)
# atario.save_syndat_model(syndat_trans6mm, os.path.join("/Users/noahwalton/research_local/resonance_fitting/ATARI_workspace/Ta181_Syndat_150eV_100keV/syndat_models_E150_100e3", "SyndatModel_6mmT.pkl"), clear_samples=True)